In [2]:
%pip install matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from pathlib import Path
from narwhals import read_csv
from sqlalchemy import create_engine
import streamlit as st
import numpy as np
import scipy
from scipy import stats

x = Path.cwd().parent
DB_PATH = x / "data" / "ab_data.csv"
table = read_csv(DB_PATH, backend="pandas",sep=",")
table = table.to_pandas()
print(table.shape)
print(table.info())
print(table.describe())
table.timestamp = pd.to_datetime(table.timestamp) #привели время к нормальному типу

(294478, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294478 entries, 0 to 294477
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294478 non-null  int64 
 1   timestamp     294478 non-null  object
 2   group         294478 non-null  object
 3   landing_page  294478 non-null  object
 4   converted     294478 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB
None
             user_id      converted
count  294478.000000  294478.000000
mean   787974.124733       0.119659
std     91210.823776       0.324563
min    630000.000000       0.000000
25%    709032.250000       0.000000
50%    787933.500000       0.000000
75%    866911.750000       0.000000
max    945999.000000       1.000000


Проведем разведочный анализ данных

In [4]:
print(table.head(6))

   user_id                  timestamp      group landing_page  converted
0   851104 2017-01-21 22:11:48.556739    control     old_page          0
1   804228 2017-01-12 08:01:45.159739    control     old_page          0
2   661590 2017-01-11 16:55:06.154213  treatment     new_page          0
3   853541 2017-01-08 18:28:03.143765  treatment     new_page          0
4   864975 2017-01-21 01:52:26.210827    control     old_page          1
5   936923 2017-01-10 15:20:49.083499    control     old_page          0


In [5]:
print(100*table["group"].value_counts(normalize=True))
print(" ")
print(100*table["landing_page"].value_counts(normalize=True))

group
treatment    50.012565
control      49.987435
Name: proportion, dtype: float64
 
landing_page
old_page    50.0
new_page    50.0
Name: proportion, dtype: float64


Видим, что не одинаковые отношения, значит есть плохие данные - очистим их.
Также видим, что перекоса в данных нет - это хорошо.
Удалим данные где группа человеку из показали некорректную для его группы страницу.
294478 - 290585 = 3893 строк удалено.

In [6]:
table = table.drop(table[(table["group"] == "control") & (table['landing_page'] ==  "new_page" )].index)
table = table.drop(table[(table["group"] == "treatment") & (table['landing_page'] ==  "old_page" )].index)
print(table.shape)

(290585, 5)


Удалено 3893 строки

In [7]:
print(len(table[table["group"] ==  "control"]))
print(len(table[table["group"] ==  "treatment"]))


145274
145311


Люди теперь распределены очень равномерно.
Теперь посмотрим как распределены пользователи и не случилось ли так, что у нас один и тот же человек попал в несколько групп

In [8]:
n = table.nunique()
print(n)
grouped = table.groupby('user_id')

user_id         290584
timestamp       290585
group                2
landing_page         2
converted            2
dtype: int64


In [9]:
duplicates = table[table.duplicated('user_id')]
print(duplicates)

      user_id                  timestamp      group landing_page  converted
2893   773192 2017-01-14 02:55:59.590927  treatment     new_page          0


In [10]:
table = table.drop(table[(table["user_id"] == 773192)].index)

In [11]:
new = table[table["group"] ==  "treatment"]
plus = len(new[new["converted"] == 1])
minus = len(new[new["converted"] == 0])
print("конверсия новой")
print(100*plus/(plus+minus))
print
old = table[table["group"] ==  "control"]
plus = len(old[old["converted"] == 1])
minus = len(old[old["converted"] == 0])
print("конверсия старой")
print(100*plus/(plus+minus))

конверсия новой
11.880888313869065
конверсия старой
12.038630450046119


H0 : Конверсия старой и новой страницы не отличаются
H1 : Конверсия старой и новой страницы отлиаются
Уровень значимости alpha = 0,05

In [12]:
!python -m pip install scipy


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
new = table[table["group"] ==  "treatment"]
old = table[table["group"] ==  "control"]
mean_new = new["converted"].mean() #среднее значенип конверсии
mean_old = old["converted"].mean()
print(f"conv_new = {mean_new}")
print(f"conv_old = {mean_old}")
n_new = len(new)
n_old = len(old)
x = table["converted"].mean()
se = np.sqrt(x*(1-x)) * np.sqrt(1 / n_new + 1 / n_old)
Z = (mean_new - mean_old) / se
print(Z)
p_value = 2*(1 - stats.norm.cdf(abs(Z)))
print(p_value)

conv_new = 0.11880888313869065
conv_old = 0.1203863045004612
-1.310240857927101
0.19011436776805035


p_value > 0.05, значит мы не можем отклонить H0
Теперь посмотрим на доверительные интервалы:

In [14]:
x_1=new["converted"].std()
x_2=old["converted"].std()
std = np.sqrt(x_1*x_1/n_new  + x_2*x_2/n_old)
diff = mean_new - mean_old
print(f"diff:{diff}")
ci_low  = diff - 1.96 * std
ci_high = diff + 1.96 * std
print(f"[{ci_low}, {ci_high}]")

diff:-0.0015774213617705535
[-0.003937101629388712, 0.0007822589058476053]


95% доверительный интервал для разницы конверсий (new − old) равен [-0.003937101629388712, 0.0007822589058476053]. Интервал включает 0, поэтому при уровне значимости 5% статистически значимого отличия между страницами не обнаружено.

В ходе A/B-теста мы сравнили конверсию пользователей на старой странице (old_page, control) и новой странице (new_page, treatment).
Конверсия в контрольной группе составила ≈ 0.1203863045004612, в тестовой — ≈ 0.11880888313869065
Абсолютная разница конверсий (new − old) составила ≈ -0.0015774213617705535, 95% доверительный интервал:  [-0.003937101629388712, 0.0007822589058476053], что включает 0.
Z-тест для двух пропорций при уровне значимости α = 0.05 не показал статистически значимого улучшения (p-value > 0.05), поэтому у нас нет оснований отклонять нулевую гипотезу и считать, что новая страница увеличивает конверсию.
На основе результатов теста не рекомендуется раскатывать новую страницу на всех пользователей как более эффективную.